# Convolutional Neural Networks

In [ ]:
!pip install pillow
!pip install opencv-contrib-python
!pip install -U fsds_100719


from fsds_100719.imports import *
import os,glob
os.path.abspath(os.curdir)

In [ ]:
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2

In [ ]:
## dataset
base_folder = '/Users/jamesirving/Google Drive/Datasets/dogs-vs-cats-sorted/'
os.listdir(base_folder)

In [ ]:
# ## DOG VS CAT
train_base_dir = base_folder+'training_set/' #r'My Drive/Datasets/dogs-vs-cats/train'
test_base_dir =base_folder+'test_set//' #r'My Drive/Datasets/dogs-vs-cats/test1'

train_dogs = train_base_dir+'dogs/'
train_cats = train_base_dir+'cats/'

test_dogs = test_base_dir+'dogs/'
test_cats = test_base_dir+'cats/'
# os.listdir(test_cats)

In [ ]:
# files = glob.glob(train_dogcat_dir+'cats/*.jpg',recursive=True)
# # files[:10]
# # os.listdir(train_dogcat_dir) 

In [ ]:
import cv2,glob,os
dog_filenames = glob.glob(train_dogs+'*.jpg')
cat_filenames = glob.glob(train_cats+'*.jpg')
img_filenames = [*dog_filenames,*cat_filenames]
print(len(img_filenames))
img_filenames[:10]

In [ ]:
def load_image_cv2(filename, RGB=True):
    """Loads image using cv2 and converts to either matplotlib-RGB (default)
    or grayscale."""
    import cv2

    IMG = cv2.imread(filename)
    if RGB: cmap = cv2.COLOR_BGR2RGB
    else: cmap=cv2.COLOR_BGR2GRAY
    return cv2.cvtColor(IMG,cmap)

In [ ]:
## Load in and display image.
IMG = load_image_cv2(img_filenames[0],RGB=False)

## Even if you import as grayscale, must tell plt to use gray cmap
fig,ax= plt.subplots(ncols=2,figsize=(12,5))
ax[0].imshow(IMG)
ax[1].imshow(IMG,cmap='gray')

## Remove axes labels https://stackoverflow.com/a/2176591
[(a.get_xaxis().set_visible(False), a.get_yaxis().set_visible(False)) for a in ax]
print(IMG.shape)

In [ ]:
## Using seaborn color palette with imshow
from matplotlib.colors import ListedColormap
cmap = ListedColormap(sns.color_palette('RdBu',n_colors=25))
plt.imshow(IMG,cmap=cmap)

In [ ]:
plt.imshow(IMG,cmap='Reds')

In [ ]:
## RESIZING IMAGES
print(IMG.shape)
small = cv2.resize(IMG,(100,50))
plt.imshow(small,cmap=cmap)

In [ ]:
## Resizing Using Ratios 
w_ratio = 0.5
h_ratio = 0.5

## Must Pass cv2.resize(IMG, (0,0) IMG, w_ratio,h_ratio)
new_img = cv2.resize(IMG, (0,0), IMG, w_ratio,h_ratio)
plt.imshow(new_img,cmap=cmap)
print(new_img.shape)

In [ ]:
new_img = cv2.flip(new_img,0)
plt.imshow(new_img,cmap=cmap)

In [ ]:
*a,_=test_base_dir.split('/')
save_dir = '/'.join(a)
save_dir

In [ ]:
cv2.imwrite(save_dir+'example_save.jpg',new_img)

In [ ]:
plt.imshow(cv2.imread(save_dir+'example_save.jpg',cv2.COLOR_BGR2RGB),cmap=cmap)
plt.gcf().patch.set_visible(False)

In [ ]:
print(base_folder)
os.listdir(base_folder)

In [ ]:
## Create training and test data
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(base_folder+'training_set/',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory(base_folder+'test_set/',
                                            target_size = (150, 150),
                                            batch_size = 32,
                                            class_mode = 'binary')


In [ ]:
training_set[0][1]

In [ ]:
shapes = ["Batchsize", "img_width","img_height","img_dim"]
SHAPES = dict(zip(shapes, training_set[0][0].shape))
print(SHAPES)
print(training_set[0][0].shape)
print('\nLabels for batch')
print(training_set[0][1])

In [ ]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(SHAPES['Batchsize'], (3, 3),
                             input_shape = (SHAPES['img_width'], SHAPES['img_height'], SHAPES['img_dim']),
                             activation = 'relu'))

classifier.add(Conv2D(SHAPES['Batchsize'], (3, 3),
                      input_shape = (SHAPES['img_width'], SHAPES['img_height'], SHAPES['img_dim']), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(SHAPES['Batchsize'], (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = SHAPES['Batchsize'], activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

# classifier.fit_generator(training_set,
#                          steps_per_epoch = 2000,
#                          epochs = 2,
#                          validation_data = test_set,
#                          validation_steps = 500,workers=-1)

In [ ]:
#https://www.kaggle.com/risingdeveloper/transfer-learning-in-keras-on-dogs-vs-cats

In [ ]:
from keras.applications import InceptionResNetV2

conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))

In [ ]:
conv_base.summary()

In [ ]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  #Sigmoid function at the end because we have just two classes
model.summary()


In [ ]:
print('Number of trainable weights before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('Number of trainable weights after freezing the conv base:', len(model.trainable_weights))

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])

In [ ]:
len(training_set)*32

In [ ]:
history = model.fit_generator(training_set,
                              steps_per_epoch = 500,
                              epochs = 2, validation_data = test_set,
                              validation_steps = 50,workers=-1)

## Overview - CNNs

CNNs have certain features that identify patterns in images because of  "convolution operation" including:

- Dense layers learn global patterns in their input feature space

- Convolution layers learn local patterns, and this leads to the following interesting features:
    - Unlike dense networks, local patterns recognized can be recognized _anywhere_ in the images. 
    - Deeper convolutional neural networks can learn spatial hierarchies.
        - A first layer will learn small local patterns, a second layer will learn larger patterns using features of the first layer patterns, etc. 
    
- Because of these properties, CNNs are great for tasks like:
    - Image classification
    - Object detection in images
    - Picture neural style transfer

## How Convolution works:

 <img src ="https://raw.githubusercontent.com/jirvingphd/dsc-04-43-02-convolutional-neural-networks-online-ds-ft-021119/master/architecture-cnn.png">
 
 - **The Convolution Operation:**
    - Detect complex biilding blocks or features that can aid in a larger task. 
        - i.e. detecting vertical or horizontal edges (example below)<br>
        <img src="https://raw.githubusercontent.com/jirvingphd/dsc-04-43-02-convolutional-neural-networks-online-ds-ft-021119/master/conv.png" width=200><br>
        - Here, the 3 x 3 matrix on the right represents the _filter_  used to perform a convolution operation to detect horizntal edges.
        - The 5x5 image on the left  would by intensity values between 1-255(or rescaled to 1-10)
    - **In Keras this is perfomed by a `Conv2d` layer.**
        - Applies the convolution filter to every possible 3x3 region posible (for above example) <br>
 <img src="https://raw.githubusercontent.com/jirvingphd/dsc-04-43-02-convolutional-neural-networks-online-ds-ft-021119/master/convolution-layer-a.png" width=400>

### **Padding:**
- Some issues arise using filteres on images.
    - Image shrinks with each convolution layer.
        - Examples:
            - Start with a 5x5 image and use a 3x3 convolution matrix, result is 3x3 image. 
            - Start with a 10x10 image and use a 3x3 conv matrix, result is 8x8 image. 
    - The pixels around the edges are used much less in the outputs due to the filter. 
- Padding solves these issues 
    - Adding one layer of pixels around the edges preserves the image size with a 3x3 filter. 
    - Example: padding our 5x5 image (so it becomes a 6x6 image), we can now use a 3x3 conv filter and the resulting image is stil 5x5.
- Padding terminology:
    - 'Valid' - no padding.
    - 'Same' - padding such that the output is the same size. 

- **Strided convolutions**
    - Stride = how the filter moves over the image. 
        - Increasing stride so that it moves 2 pixels over instead of 1 results in a smaller # of locations.
        - Rarely used in practice, but important option for some models. 
 
### Convolutions with color images: 
- Let's assume a 7x7 RGB, which becomes a 7 x 7 x 3 tensor. 
    - Will need to use a filter that has the third dimension as 3. 
    - Can use to detect features on individual color channels.
- Can deconvolve with several 3D filters, then stack the output results together. 

- **Advantage: your image can be huge and the amount of parameters only depends on how many _filters_ used.**
    - Lets say we have 20 3x3x3(color) images 
        - Would have $20 * 27 + a_bias_for_each_filter (1*20) = 560$ parameters

- **Notation:**

    - $f^{[l]}$ = size of the filter
    - $p^{[l]}$ = padding
    - $s^{[l]}$ = amount of stride
    - $ n_c^{[l]}$ = number of filters
    - filter: $f^{[l]}$ x $f^{[l]}$ x $ n_c^{[l-1]}$


- Input =  $n_h^{[l-1]} * n_w^{[l-1]} * n_c^{[l-1]}$
- Output = $n_h^{[l]} * n_w^{[l]} * n_c^{[l]}$

- Height and width are given by:

    - $n_h^{[l]}= \Bigr\lfloor\dfrac{n_h^{[l-1]}+2p^{[l]}-f^{[l]}}{s^{[l]}}+1\Bigr\rfloor$

    - $n_w^{[l]}= \Bigr\lfloor\dfrac{n_w^{[l-1]}+2p^{[l]}-f^{[l]}}{s^{[l]}}+1\Bigr\rfloor$


- Activations: $a^{[l]}$ is of dimension $ n_h^{[l]} * n_w^{[l]} * n_c^{[l]} $

## Pooling Layers
- After Conv layers, intended to significantly downsample the previous convolution layer.
    - After conv layers detect edges/patterns, poooling layers will take a summary of the convoltuions from a larger section.
    - Most common pooling layer is MaxPooling
        - Takes the maximum of all convolutions from a larger area of the iamge. 
        - Works better than average pooling.
- Downsampling is essential for viable executin times. 

- **MaxPooling Hyperparameters:**
    - Parameter names:
        - filter_size (f)
        - stride (S)
    - Common combinations:
        - f=2, s=2
        - f=3, s=2
        
### Additional Resources

* https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html
* https://datascience.stackexchange.com/questions/16463/what-is-are-the-default-filters-used-by-keras-convolution2d
* https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-convolutional-neural-networks
* https://www.coursera.org/learn/convolutional-neural-networks/lecture/A9lXL/simple-convolutional-network-example
* https://www.coursera.org/learn/convolutional-neural-networks/lecture/uRYL1/cnn-example

In [ ]:
# from keras import layers
# from keras import models
# from keras import optimizers
# import datetime

# original_start = datetime.datetime.now()
# start = datetime.datetime.now()

# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu',
#                         input_shape=(150, 150, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dense(3, activation='softmax'))


# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.RMSprop(lr=1e-4),
#               metrics=['acc'])

# history = model.fit_generator(train_generator,
#                               steps_per_epoch=100,
#                               epochs=10,
#                               validation_data=validation_generator,
#                               validation_steps=50)

# end = datetime.datetime.now()
# elapsed = end - start
# print('Training took a total of {}'.format(elapsed))

### Using os, shutil to create directories and copy files
- from [Convolutional Neural Networks - Codealong](https://github.com/jirvingphd/dsc-04-43-03-convolutional-neural-networks-code-along-online-ds-ft-021119)

- **first define the folders that currently contain the images get their filenames**

```python
import os, shutil

# Define directories to be created:
data_santa_dir = 'data/santa/'
data_not_santa_dir = 'data/not_santa/'
new_dir = 'split/'

# Store the list of all the relevant training target images
imgs_santa = [file for file in os.listdir(data_santa_dir) if file.endswith('.jpg')]
print('There are',len(imgs_santa), 'santa images')

# Store the list of all non-target images
imgs_not_santa = [file for file in os.listdir(data_not_santa_dir) if file.endswith('.jpg')]
print('There are', len(imgs_not_santa), 'images without santa')

```

- **Now create new directries and for training, testing, and validation images.**

```python
# Create the main folder for all of the new sub-folders
os.mkdir(new_dir)

# Create valid pathnames inside of the new_dir for training images
train_folder = os.path.join(new_dir, 'train')
train_santa = os.path.join(train_folder, 'santa')
train_not_santa = os.path.join(train_folder, 'not_santa')

# Create valid pathnames inside of the new_dir for testing images
test_folder = os.path.join(new_dir, 'test')
test_santa = os.path.join(test_folder, 'santa')
test_not_santa = os.path.join(test_folder, 'not_santa')

# Create valid pathnames inside of the new_dir for validation images
val_folder = os.path.join(new_dir, 'validation')
val_santa = os.path.join(val_folder, 'santa')
val_not_santa = os.path.join(val_folder, 'not_santa')


# Now create all of the folders defined above
os.mkdir(test_folder)
os.mkdir(test_santa)
os.mkdir(test_not_santa)

os.mkdir(train_folder)
os.mkdir(train_santa)
os.mkdir(train_not_santa)

os.mkdir(val_folder)
os.mkdir(val_santa)
os.mkdir(val_not_santa)

```

- **Now that we have the folders, copy the desired # of images to the correct dataset folders**

```python
# The user decided to put 271 images in the training set, 100 in the validation set, and 90 in the test set
# train santa
imgs = imgs_santa[:271]
for img in imgs:
    origin = os.path.join(data_santa_dir, img)
    destination = os.path.join(train_santa, img)
    shutil.copyfile(origin, destination)
    
# validation santa
imgs = imgs_santa[271:371]
for img in imgs:
    origin = os.path.join(data_santa_dir, img)
    destination = os.path.join(val_santa, img)
    shutil.copyfile(origin, destination)
    
# test santa
imgs = imgs_santa[371:]
for img in imgs:
    origin = os.path.join(data_santa_dir, img)
    destination = os.path.join(test_santa, img)
    shutil.copyfile(origin, destination)
    
## REPEATED FOR FOR THE NON-SANTA IMAGES - NOT SHOWN   
```

- Now that we have images in separate directories, we can use the Kera's ImageDataGenerators .flow_from_directory() method.


```python

# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64, 64), batch_size = 180) 
# ...do the same for train and val (not shown)


# create the data sets
train_images, train_labels = next(train_generator)

# Make sure things worked
print ("Number of training samples: " + str(m_train))
print ("train_images shape: " + str(train_images.shape))


# Reshape the training images to have one column(/row?) for each image
train_img = train_images.reshape(train_images.shape[0], -1)
print(train_img.shape)


# Reshape the labels to match the data
train_y = np.reshape(train_labels[:,0], (542,1))

```

## Building CNN From Scratch Lab
- https://github.com/learn-co-students/dsc-04-43-04-building-a-cnn-from-scratch-online-ds-ft-021119/tree/solution
- CNN's are great for image processing
### Image Data

```python
import os #for listdir()
from keras.preprocessing.image import ImageDataGenerator


train_dir = 'chest_xray_downsampled/train'
validation_dir = 'chest_xray_downsampled/val/'
test_dir = 'chest_xray_downsampled/test/'

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train_generator example
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

```
- **Images are store in ImageDataGenerators**
    - Generally rescale to... intensity values? of 1./255
    - load in files with ImageDataGenerator.flow_from_directory( :
        - directory
        - the target_size (the size to convert all images to)
        - batch_size
        - class_mode 
            - Note: selection of loss function determines chocie.
                - If using 'binary_crossentropy' for binary classification, use class_mode='binary'
                
- ImageDataGenerators are also used for augmenting data. 

```python

train_datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

```

### Setting Up Initial Network
```python
from keras import models, layers, optimizers
# or if want to do exact layers:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense)
from keras.optimizers import RMSprop

# Initialize sequential model
model = Sequential()
```
**1A) A CNN should start with a Conv2D later**

```python
layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
```

- Conv2D layers parameters (to change):
    - filters:  # of samples to take from each image (kind of like # of neurons?) (e.g. filters=32)
    - kernel_size: size (in pixels) of the filters (e.g kernel_size=(3,3)
    - activation: activation function to use (e.g. 'relu')
        
**1B) MaxPooling2D layers following Conv2D layers**
```python
layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)
```

-  MaxPooling2D parameters:
    -  pool_size: factor by which to downscale. 
        - e.g.pool_size=(2,2) will half information in vertical and horizational direction
     
**1C,optional) Add a Dropout layer to avoid overfitting:** [Udemy course suggestion]
```python
layers.Dropout(rate, noise_shape=None, seed=None)
```
- Dropout parameters:
    - rate = 0.25 (used by udemy course)

**2) Repeat: Continue layering combinations of Conv2D / MaxPooling2D layers** (Dropout too?):
- Later layers will need larger # of filters to detect more abstract patterns.

```python
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
```

**3A) Flatten Data Before Passing on to Dense Layers for Classification/Learning**
```python
layers.Flatten(data_format =None)
```
**3B)  Add Dense layers at the end of the convolutional base for learning:**
```python
layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
```
- Will only need to worry about basic parameters:
    - Units:
        - Larger #, used for the actual learning.
    - Activation
        - User choice, 'relu' is always good.
        
**3C) Add final Dense layer to determine output classification**
- Add  a final small Dense layer (depending on number of classes?)
    - For binary classification:
        - units: 1
        - activation: 'sigmoid'
        
``` python
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
```

**4) Compile the model, selecting loss function, optimizer, and metric**

- Loss Function:
    - For binary classifications, use 'binary_crossentropy'
- Optimizer:
    - Use RMSProp,
        -Specify learning rate: ```lr = 1e-4```
- Metrics:
    - Use 'acc' for accuracy
```python
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
```


```python
# Compile Model
model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.RMSProp(lr=1e-4),
              metrics=['acc'])

#Set the model to train; 
import datetime
start=datetime.datetime.now()
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

end=datetime.datetime.now()
```

# Use Pretrained CNNs

## Pretrained Networks Overview
- Pretrained networks have already been trained on large pools of data, and have their weights frozen.
    - They enable deep learning on fairly small image datasets
    - a 'small' dataset is less than tens of thousands or hundreds of thousands of images

- Pretrained networks can be used in whole or only specific parts, depending on your need/data.
    - The shallower the layer of neurons, the more generic its features are. 
        - Therefore even if you data is very different, you can still use the lower layers for basic feature extraction.
    - The deeper the layer, the more abstract its features are.
        - so you may want to unfreeze the deeper/higher order classificaiton layers and re-train the network on your images. 
<br><br>
### Where to find the pre-trained networks
- **Pretrained Networks are available in [Keras.applications](https://keras.io/applications/)**
    - This list of pretained models are for image classification. 
        - DenseNet
        - InceptionResNetV2
        - InceptionV3
        - MobileNet
        - NASNet
        - ResNet50
        - VGG16
        - *VGG19* - used in labs
        - Xception

    - You can import these networks and use it as a function with 2 arguments:
        1. `weights`
            - Determines which data source's training data weights to use.
            - ex:  `weights='imagenet'
        2. `include_top`
            - determines whetehr or not to include the fully-connected layer at the top of the network
```python
from keras.applications import MobileNet
conv_base = MobileNet(weights='imagenet', include_top=True)
```

### How to use pretrained networks for feature extraction or for fine-tuning

**You'll learn about two ways to use pre-trained networks:**
- **Feature extraction**: here, you use the representations learned by a previous network to extract interesting features from new samples. 
    - Method 1) Use the convolutional base layers and run your data to detect the basic features, and save the output data, which is then run a new dense classifier, which is trained from scratch.  
        - (+) It is fast
        - (-) but cant use data augmentation. 
        - Note:  If your images are very different from the pretraining datasets, you may want to only use _part_ of the convolutional base but a _new_ densely connected classifier
    - Method 2) Extend the conv_base by adding dense layers on top, running everything together. 
        - (+) allows for data sugmentation
        - (-) extremely time-consuming and requires GPU
  
- **Fine-tuning**: when finetuning, you'll "unfreeze" a few top layers from the convolutional base of the model and train them again together with the densely connected classifier layers of the model. 
    - Note that you are changing the parts of the convolutional layers here that were used to detect the more abstract features.
    - By doing this, you can make your model more relevant for the classification problem at hand.

## Additional Resources

* http://cs231n.stanford.edu/syllabus.html
* https://www.dlology.com/blog/gentle-guide-on-how-yolo-object-localization-works-with-keras/
* https://www.dlology.com/blog/gentle-guide-on-how-yolo-object-localization-works-with-keras-part-2/


In [ ]:
from keras.applications import MobileNet
conv_base = MobileNet(weights='imagenet',
                  include_top = True)
conv_base.summary()

# Using  Pretrained Networks - Codealong
## Theory/Tips

## Code 

### Feature Extraction Method 1:

```python
from keras.applications import VGG19
cnn_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(64, 64, 3))
cnn_base.summary()

# ---

def extract_features(directory, sample_amount):
    features = np.zeros(shape=(sample_amount, 2, 2, 512)) 
    labels = np.zeros(shape=(sample_amount))
    generator = datagen.flow_from_directory(
        directory, target_size=(64, 64), 
        batch_size = 10, 
        class_mode='binary')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = cnn_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch 
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i = i + 1
        if i * batch_size >= sample_amount:
            break
    return features, labels

# ---

# you should be able to divide sample_amount by batch_size!!
train_features, train_labels = extract_features(train_folder, 540) 
validation_features, validation_labels = extract_features(val_folder, 200) 
test_features, test_labels = extract_features(test_folder, 180)

train_features = np.reshape(train_features, (540, 2 * 2 * 512))
validation_features = np.reshape(validation_features, (200, 2 * 2 * 512))
test_features = np.reshape(test_features, (180, 2 * 2 * 512))

# ---

from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=2 * 2 * 512))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(train_features, train_labels,
                    epochs=20,
                    batch_size=10,
                    validation_data=(validation_features, validation_labels))

results_test = model.evaluate(test_features, test_labels)

# ---

train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epch = range(1, len(train_acc) + 1)
plt.plot(epch, train_acc, 'g.', label='Training Accuracy')
plt.plot(epch, val_acc, 'g', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.figure()
plt.plot(epch, train_loss, 'r.', label='Training loss')
plt.plot(epch, val_loss, 'r', label='Validation loss')
plt.title('Loss')
plt.legend()
plt.show()

#---

```

## Feature Extraction Method 2
 - this method is much more costly, but allows us to use data augmentation
 
- The process:
    1. Add the pretrained model as the first layer
    2. Add some dense layers as a classifier on top
    3. Freeze the convolutional base
        - This will prevent the weights from changing. 
        - The layer.trainable attribute indicates if a layer is frozen
    4. Train the model. 
    
    
```python

model = models.Sequential()
model.add(cnn_base)
model.add(layers.Flatten())
model.add(layers.Dense(132, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# ---

#You can check whether a layer is trainable (or alter its setting) through the layer.trainable attribute:
for layer in model.layers:
    print(layer.name, layer.trainable)
    
#Similarly, we can check how many trainable weights are in the model:
print(len(model.trainable_weights))

# ---

# Freeze the conv base
cnn_base.trainable = False


# ---

# get all the data in the directory split/train (542 images), and reshape them
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
        batch_size= 20,
        class_mode= 'binary') 

# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_folder, 
        target_size=(64, 64), 
        batch_size = 20,
        class_mode= 'binary')

# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64, 64), 
        batch_size = 180,
        class_mode= 'binary')

test_images, test_labels = next(test_generator)

# ---

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

# ---

history = model.fit_generator(
              train_generator,
              steps_per_epoch= 27,
              epochs = 10,
              validation_data = val_generator,
              validation_steps = 10)

```



## Fine Tuning

Up till now, we have frozen the entire convolutional base. Again, it cannot be stressed enough how important this is before fine tuning the weights of the later layers of this base. Without training a classifier on the frozen base first, there will be too much noise in the model and initial epochs will overwrite any useful representations encoded in the pretrained model. That said, now that we have tuned a classifier to the frozen base, we can now unfreeze a few of the deeper layers from this base and further fine tune them to our problem scenario. In practice, this is apt to be particularly helpful where adapted models span new domain categories. For example, if the pretrained model is on cats and dogs and this is adapted to a problem specific to cats (a very relatively similar domain) there is apt to be little performance gain from fine tuning. On the other hand, if the problem domain is more substantially different, additional gains are more likely in adjusting these more abstract layers of the convolutional base. With that, let's take a look at how to unfreeze and fine tune these later layers.



```python

cnn_base.trainable = True

# ---

cnn_base.trainable = True
set_trainable = False
for layer in cnn_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
        
# ---

model.compile(loss='binary_crossentropy',
                      optimizer=optimizers.RMSprop(lr=1e-4),
                      metrics=['accuracy'])

# ---
history = model.fit_generator(
              train_generator,
              steps_per_epoch= 27,
              epochs = 10,
              validation_data = val_generator,
              validation_steps = 10)


# NLP Content from Flation Data Science Bootcamp

## Word Embeddings Lab


- [Solution on Github](https://github.com/jirvingphd/dsc-04-45-04-generating-word-embeddings-lab-online-ds-ft-021119/tree/solution)

- Use `nltk.word_tokenize` to tokenize new headlines data
    - Can use `dataframe['column'].map(word_tokenize)` to tokenize a specific column in a df.
    - After tokenization, leave in original order. 
    
- Use `gensim.models.Word2Vec`
    - [gensim website](https://radimrehurek.com/gensim/)
    
    
- Instantiate a Word2Vec model:<br> `model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)`
    - `data` = text
    - `size` =size of the embedding vectors to create
    - `window` = # of words to include in sliding window
    - `min_count` = number of times a word must appear to be counted
    - `wokers` = number of threads to use during training
    
- `model.train(data, total_examples = model.corpus_count, epochs=10)`

- Now can use the model.wv dictionary for methods
    - `wv = model.wv`
    
    
- **Get Word Similarity**
    - Return the Most Similar Words 
        - `wv.most_similar('Texas')`
         
    - Return the Least Similar Words (not so meaningful)
        - `wv.most_similar(negative='Texas')`

- **To Get a Word's Vector**
    - Use wv as a dictionary
      - `wv['Texas']`
      
- **To Get All Word Vectors**
    - `wv.vectors`
        
- **To Perform Word *Arithmetic***
    - i.e. 'king' - 'man' + 'woman'
    - Words to add should be `positive=`, words to subtract are `negative=`
    - `wv.most_similar(positive=['king','woman'], negative=['man'])`

In [ ]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
from nltk import word_tokenize

import nltk
nltk.download('punkt')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

file = '/content/gdrive/My Drive/Colab Notebooks/datasets/News_Category_Dataset_v2.json'
df = pd.read_json(file, lines=True)
df.head()

In [ ]:
# Concatenate description and headline.
df['combined_text'] = df.headline+' '+df.short_description

# Tokenize the combined_text column.
data = df['combined_text'].map(word_tokenize)

# Preview first 5
data[:5]

In [ ]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

model.train(data, total_examples=model.corpus_count, epochs=10)


wv = model.wv
wv.most_similar('Texas')

In [ ]:
wv.most_similar(negative='Texas')

In [ ]:
wv.most_similar(positive=['king','woman'], negative=['man'])

___

## Classification with Word Embeddings

### Using Pretrained Word Vectors with GloVe
- Best to load a top-tier industry-standard word mdels
    - Most common is Global Vectors for Word Representation (GloVe) from the Stanford NLP Group.
    - Loading in weights removes the need to instantiate a Word2Vec model. 
- **Instead, the process is:**
    1. Get the total vocabulary of our data
    2. Download and unzip the GloVe file needed from Standford NLP
    3. Read the GloVe file, save only vectors for words in our dataset.
    

    
- **File must be downloaded manually:**
    - Must download the zip file for the Stanford Groups pretrained vectors [here.](https://nlp.stanford.edu/projects/glove/)(Lab used the smallest one (which is still 6 B words)
    - Place the downloaded file directly into same folder as jupyter notebook.
- **To use in python;**    
    - First, must tokenize the words in the vocab as done above with nltk.
    - Second, must turn the vocabulary into a `set`:
        `total_vocabulary = set(word for headline in data for word in headline)`
    - Third, load in the glove file, check for and keep only the words that are inside of total_vocabulary
```python
glove = {}
with open('glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector
```
- The code above has created a dictionary called `glove, which contains all of the vectors for our data's vocabulary.

### Mean Word Embeddings

- Just loading in vectors does not describe sentence, only individual words.
- To classify text, we need to calculate ***Mean Word Embeddings***. 
    - Simply get the vector for every word in a sentence and take the average.
    - Mean Word vectors will always match the size of each individual word, no matter how many words appear in a sentence. 
    - Can easily put text into a form for Supervised Learning models, such as Support Vector Machines or Gradient Boosted Trees. 
    
    
### Coding a Custom Vectorizer Class (compatible with sklearn)

### Deep Learning & Embedding Layers

- Mean word embeddings lose some of the meaning, which is why **Sequence Models** exist. 
    - Recurrent Neural Networks 
    - Long Short Term Memory Cells
- For deep learning, add **Embedding Layers** into the network.

- **Embedding Layer Requirements**
    - Learn the word embeddings for our data 'on the fly', get the benefits of Word2Vec without needing to train a Word2Vec model separately.
    - Embedding Layers must always be the FIRST layer, immediately below the Input() layer.
    - All words in the text must be integer-encoded (each word its own unique integer)
    - Size of the embedding layer MUST be greater than the total vocabulary size
        - First parameter denotes vocab size, the second parameter denotes word vector size. 
    - The size of sequences passed must be set when creating the layer. 
    
###  Embedding Layers in Keras
- [Classification with Word Embeddings - Lab ](https://github.com/learn-co-students/dsc-04-45-06-classification-with-word-embeddings-lab-online-ds-ft-021119)
- [Keras Documentation](https://keras.io/layers/embeddings/)

```python
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

y = pd.get_dummies(target).values
```
- **Preprocessing data for use with embedding layer:**
    - Tokenize each example
    - Convert to sequences
    - Pad the sequences so same length

```python
tokenizer = text.Tokenizer(num_words=20000) # limiting to first 20000 words in vocab
tokenizer.fit_on_texts(list(df.combined_text))
list_tokenized_headlines = tokenizer.texts_to_sequences(df.combined_text)
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

```

- **Setting up the network architecture:**
    - Input layer first
    - Embedding layer second
        - pass size of vocab, embedding_size
        - embedding_size = 128
    - LSTM layer third
        - Followed by a GlobalMaxPooling1D layer
        - Followed by  Dropout layer
    - Dense layer for classification (activation='relu')
        - Followed by another Dropout layer
    - Final Dense layer
        - Number of neurons = # of possible classes.
        - activation = 'softmax'

```python

embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(20000, embedding_size)(input_)
x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 41 different possible classes, so we use 41 neurons in our output layer
x = Dense(41, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_t, y, epochs=2, batch_size=32, validation_split=0.1)


```


### Lab's W2vVectorizer Class (edited for bs_ds compatibility)
- Original did not have import numpy statements and did not accept glove during \_\_init__ (but still expected it to be present)
```python
class W2vVectorizer(object):
    """From Learn.co Text Classification with Word Embeddings Lab.
    An sklearn-comaptible class containing the vectors for the fit Word2Vec."""

    def __init__(self, w2v, glove):
        # takes in a dictionary of words and vectors as input
        import numpy as np

        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])

    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline.
    def fit(self, X, y):
        return self

    def transform(self, X):
        import numpy as np
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])
```

#### With W2vVectorizer, can use in sklearn Pipelines:

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

# ---
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]
# ---
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]
scores
# ---
```
